# SpaceBandits Contextual Bandits Demo
Этот блокнот демонстрирует базовое использование SpaceBandits. Пакет в настоящее время находится в разработке. Установить с помощью:

```
pip install space_bandits
```

## Построить линейную модель
Самая простая модель в пакетах сопоставляет контексты с ожидаемыми вознаграждениями с линейными коэффициентами. Используйте функцию конструктора модели; необходимо указать длину признака (количество признаков в строке) и количество доступных действий.

In [1]:
from space_bandits import LinearBandits

num_actions = 5 #five actions
num_features = 10 #ten features

model = LinearBandits(num_actions, num_features)

## Обучите модель с помощью метода .update()
Используйте прошлые примеры контекста, действия, вознаграждения для обучения модели. Контекст должен иметь указанное выше измерение; каждый пример обучения должен включать одно действие (индексированное с нуля) и одно связанное вознаграждение.

In [2]:
import numpy as np
context = np.random.random((10))
print('пример вектора контекста: \n', context)
action = 2
print('пример выбранного действия: \n', action)
reward = 5
print('пример вознаграждения: \n', reward)

#здесь мы обновляем модель:
model.update(context, action, reward)

example context vector: 
 [0.85174531 0.14239978 0.01099373 0.90709505 0.67988471 0.6440062
 0.21293169 0.09332591 0.6603089  0.78953261]
example action chosen: 
 2
example reward associated with: 
 5


## Принимайте решения с помощью метода .action()

После обучения модели мы можем использовать метод .action() для сопоставления заданного контекста с действием с наивысшим ожидаемым вознаграждением.

In [3]:
new_context = np.random.random((10))
print('новый пример вектора контекста: \n', context)

print('модель предложила действие: ')
print(model.action(new_context))

new example context vector: 
 [0.0425226  0.43738244 0.07786224 0.25217616 0.91765093 0.19091702
 0.92737656 0.29993496 0.45324536 0.08770382]
model suggested action: 
1


## Ожидаемые значения
Мы можем напрямую вывести ожидаемые значения модели для каждого действия, учитывая контекст, используя метод expected_values(). Неопределенность ожидаемых значений не принимается во внимание в этом вычислении.

In [3]:
print('ожидаемые значения для примера контекста: ')
print(model.expected_values(context))

ожидаемые значения для примера контекста: 
[0.         0.         4.74007592 0.         0.        ]


## Advanced Parameters
### Memory Management
The model keeps a record of all previous examples; this is useful for updating, but it's impractical in ongoing production scenarios. To limit the model's memory, specify the number of previous examples to "remember" using the memory_size argument.

```python
model = init_linear_model(num_actions, context_dim, memory_size=1000000)
```

The above specifies that the model only keep a running record of the last 1000000 examples.

### Initial Exploration
Thompson sampling gives us continuous, intelligent exploration throughout the model's lifetime. However, initial exploration can be very helpful for encouraging model convergence, especially with a cold start. Use the initial_pulls argument to force the model to explore before defaulting to Thompson sampling. The model will sequentially try each action initial_pulls number of times; this results in initial_pulls * n_actions exploratory actions.

```python
model = init_linear_model(num_actions, context_dim, initial_pulls=2)
```

The above will result in the model suggesting each action 2 times before using Thompson sampling to suggest actions.

### Saving Your Model
Each SpaceBandits model has a .save() method. Use it to save models for later use.

In [5]:
model.save('my_saved_model') #save to file my_saved_model

from space_bandits import load_model

model = load_model('my_saved_model') #load from same location

## Building a Neural Model

Linear models are powerful but inherently limited. The Neural-Linear Bayesian Contextual Bandits model, which was named and explored in the 2018 research paper [Deep Bayesian Bandits Showdown: An Empirical Comparison of Bayesian Deep Networks for Thompson Sampling](https://arxiv.org/pdf/1802.09127.pdf), uses a neural network to give the model a powerful way to map a feature vector to a latent representational feature space. These learned features are used in a standard linear model identical to the one used above.<br><br>
SpaceBandits lets us deploy the same model with the API as above. In practice, designing the model is can be somewhat complicated; the neural network adds a huge number of hyperparameters. SpaceBandits uses the default parameters used in the research paper to give users a nice starting point; modifying them is easy.

In [6]:
from space_bandits import NeuralBandits

model = NeuralBandits(num_actions, num_features)

We can update the model in the same way as before. To improve training efficiency, the neural network only trains after a pre-defined number of updates. The default neural network training frequency is every 50 updates (modify the training_freq_network argument to change this); each time this occurs, the network trains for 100 epochs at each training session by default (modify the training_epochs argument to change this).

In [7]:
# here we update the model 100 times.

for i in range(100):
    context = np.random.random((10))
    action = np.random.randint(0, 4)
    reward = np.random.random() * 10
    model.update(context, action, reward)

Training neural_model-bnn for 100 steps...
Training neural_model-bnn for 100 steps...


As with the linear model, the neural model will record all examples by default; modify the memory_size parameter (default value -1, for inf) on the constructor function to manage memory and training time.

## Saving a Neural Model

Neural models actually consist of two models: a neural network and a Bayesian linear regression model. To manage this for saving, SpaceBandits creates a .zip file that keeps your models together.

In [8]:
model.save('my_neural_model.pkl')

#don't forget the .zip extension when restoring your neural model.
model = load_model('my_neural_model.pkl')

## Expected Values
We don't like black boxes. Model interpretation is critical for solid data science. Any SpaceBandits model will return its expected reward values for a given context using the .expected_values() method:

In [9]:
model.expected_values(context)

array([2.01993425, 5.70811732, 2.41866919, 5.86543991, 0.        ])

Neural models make use of a latent representation of the input features; this feature vector is called $z$ in the Google Brain research paper. You can retrieve the model's latent feature vector using the .get_representation() method.

In [10]:
model.get_representation(context)

tensor([ 2.1710,  0.0000,  0.0000,  1.8167,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000, 13.5386,  3.6377,  0.0000,  0.0000,  8.4940,  0.0000,
         2.7854,  7.9246,  0.0000,  0.0000,  8.4467,  0.0000,  0.0000,  0.0000,
         9.4823,  0.0000,  7.0178,  5.1963,  2.3382,  5.0501,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.4022,  0.0000, 11.3781,  0.0000,  0.0000,  0.0000,  0.3589,  0.0000,
         6.1590,  0.0000])

This 50-dimensional vector encodes the useful information in a context for our linear model to use. While it may not seem particularly useful at this time, remember: SpaceBandits is from the future, and we know this stuff turns out to be important ;)<br><br>
By default, the neural network has one hidden layer with 50 nodes. Use a list of integers in the layer_sizes argument to the constructor to specify number of layers and nodes.

## Model Evaluation
Evaluating bandit models in real-life situations is not easy. The only way to really tell if your model is doing well is to put it into production and compare its results to other decision-making policies. Simulations and toy problems where action/reward relationships are known are a great place to start. Unfortunately, public contextual bandits datasets are hard to come by!<br><br>
For a look at some toy problems, check out the [toy problem notebook](toy_problem.ipynb).